In [ ]:
!pip install psaw

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install praw

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 188 kB 5.3 MB/s 
     |████████████████████████████████| 55 kB 3.5 MB/s 


In [ ]:
!pip install config

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Find most trending stocks in Reddit

In [ ]:
#  Import Libraries
import pandas as pd
import praw
import time
import matplotlib.pyplot as plt
from datetime import date
import datetime as dt
import plotly.express as px
import numpy as np

In [ ]:
stocks = pd.read_csv('..\data\meta\all_tickers.csv')

In [ ]:
# Last Sale column is an object, let's convert to float
stocks['Last Sale'] = stocks['Last Sale'].str.replace('$', '')
stocks['Last Sale'] = pd.to_numeric(stocks['Last Sale'], downcast='float')
type(stocks['Last Sale'][0])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  


numpy.float32

In [ ]:
# Filter out stocks >$3 and > $100 million cap
price_filter = stocks['Last Sale'] >= 3.00
cap_filter = stocks['Market Cap'] >= 100000000
# make set of symbols
stocks = set(stocks[(price_filter) & (cap_filter)]['Symbol'])

In [ ]:
# remove nan 
stocks = {x for x in stocks if x==x}

In [ ]:
# Add in well known meme stocks into the list that we may have missed out
stocks = stocks.union(set(['GME', 'AMC', 'PLTR', 'NVDA', 'F', 'BB', 'NOK', 'SPCE', 'WISH', 'RBLX', 'CLOV', 'ROKU', 'GO', 'SOFI', 'FUBO', 'WKHS']))

In [ ]:
print("\nRunning sentiment analysis, this may take a few minutes...\n")

# Instantiate praw object
start_time = time.time()
reddit = praw.Reddit(
    user_agent = "Stocks Extraction",
    client_id = CLIENT_ID ,
    client_secret = SECRET_KEY,
    check_for_async=False
)

# Set program parameters
subs = ['wallstreetbets', 'stocks', 'investing', 'stockmarket']     # sub-reddit to search
blacklist = {'I', 'ELON', 'WSB', 'THE', 'A', 'ROPE', 'YOLO', 'TOS', 'CEO', 'DD', 'IT', 'OPEN', 'ATH', 'PM', 'IRS', 'FOR','DEC', 'BE', 'IMO', 'ALL', 'RH', 'EV', 'TOS', 'CFO', 'CTO', 'DD', 'BTFD', 'WSB', 'OK', 'PDT', 'RH', 'KYS', 'FD', 'TYS', 'US', 'USA', 'IT', 'ATH', 'RIP', 'BMW', 'GDP', 'OTM', 'ATM', 'ITM', 'IMO', 'LOL', 'AM', 'BE', 'PR', 'PRAY', 'PT', 'FBI', 'SEC', 'GOD', 'NOT', 'POS', 'FOMO', 'TL;DR', 'EDIT', 'STILL', 'WTF', 'RAW', 'PM', 'LMAO', 'LMFAO', 'ROFL', 'EZ', 'RED', 'BEZOS', 'TICK', 'IS', 'PM', 'LPT', 'GOAT', 'FL', 'CA', 'IL', 'MACD', 'HQ', 'OP', 'PS', 'AH', 'TL', 'JAN', 'FEB', 'JUL', 'AUG', 'SEP', 'SEPT', 'OCT', 'NOV', 'FDA', 'IV', 'ER', 'IPO', 'MILF', 'BUT', 'SSN', 'FIFA', 'USD', 'CPU', 'AT', 'GG', 'Mar','VERY','ON','HAS','LOVE','IQ','FA','HA','BOOM','GO','CAN','IBKR','FUND','POW','GOOD','CENT','GOOG'}
blacklist.update(['HOOD', 'SP', 'DWAC', 'TOP', 'RUM', 'STEP', 'CSQ', 'ORGN', 'LAX'])
goodAuth = {'AutoModerator'}   # authors whom comments are allowed more than once
uniqueCmt = True                # allow one comment per author per symbol
upvoteRatio = 0.70         # upvote ratio for post to be considered, 0.70 = 70%
ups = 20       # define # of upvotes, post is considered if upvotes exceed this #
limit = 10      # define the limit, comments 'replace more' limit
upvotes = 2     # define # of upvotes, comment is considered if upvotes exceed this #
picks = 100     # define # of picks here, prints as "Top ## picks are:"
picks_ayz = 100   # define # of picks for sentiment analysis

posts, count, c_analyzed, tickers, titles, a_comments = 0, 0, 0, {}, [], {}
cmt_auth = {}

for sub in subs:
    subreddit = reddit.subreddit(sub)
    hot_python = subreddit.top(time_filter = 'year', limit = 250) # sorting posts by hot
    # Extracting comments, symbols from subreddit
    for submission in hot_python:
        try:
            author = submission.author.name
        except:
            pass

        # Checking: post upvote ratio # of upvotes, post flair, and author
        if submission.upvote_ratio >= upvoteRatio and submission.ups > ups:
            submission.comment_sort = 'new'
            comments = submission.comments
            titles.append(submission.title)
            posts += 1
            submission.comments.replace_more(limit = limit)
            for comment in comments:
                # try except for deleted account?
                try:
                    auth = comment.author.name
                except:
                    pass
                c_analyzed += 1

                # checking: comment upvotes and author
                if comment.score > upvotes:
                    split = comment.body.split(' ')
                    for word in split:
                        word = word.replace("$", "")
                        # upper = ticker, length of ticker <= 5, excluded words
                        if word.isupper() and len(word) <= 5 and word not in blacklist and word in stocks:
        
                            # unique comments, try/except for key errors
                            if uniqueCmt and auth not in goodAuth:
                                try:
                                    if auth in cmt_auth[word]:
                                        break
                                except:
                                    pass
                            # counting tickers
                            if word in tickers:
                                tickers[word] += 1
                                a_comments[word].append(comment.body)
                                cmt_auth[word].append(auth)
                                count += 1
                            else:
                                tickers[word] = 1
                                cmt_auth[word] = [auth]
                                a_comments[word] = [comment.body]
                                count += 1

# sorts the dictionary
symbols = dict(sorted(tickers.items(), key=lambda item: item[1], reverse = True))
top_picks = list(symbols.keys())[0:picks]
time = (time.time() - start_time)

# print top picks
print("It took {t:.2f} seconds to analyze {c} comments in {p} posts in {s} subreddits.\n".format(t=time,
                                                                                                c=c_analyzed,
                                                                                                p=posts,
                                                                                                s=len(subs)))


Running sentiment analysis, this may take a few minutes...

It took 12367.29 seconds to analyze 210622 comments in 995 posts in 4 subreddits.



In [ ]:
top_picks

['GME',
 'TSLA',
 'AMD',
 'NVDA',
 'BBBY',
 'AAPL',
 'MSFT',
 'AMC',
 'AMZN',
 'F',
 'PLTR',
 'WISH',
 'NFLX',
 'TA',
 'ATVI',
 'PTON',
 'PYPL',
 'COIN',
 'DKNG',
 'INTC',
 'META',
 'RIVN',
 'CLOV',
 'GOOGL',
 'VC',
 'Z',
 'BB',
 'CRSP',
 'MRNA',
 'JD',
 'SOFI',
 'ASML',
 'RBLX',
 'ZM',
 'EA',
 'LCID',
 'FUBO',
 'TTD',
 'NKLA',
 'WBD',
 'CD',
 'COST',
 'MNMD',
 'BASE',
 'NOK',
 'RICK',
 'FANG',
 'AFRM',
 'EVER',
 'FREE',
 'TTWO',
 'TTCF',
 'BYND',
 'PLUS',
 'BLUE',
 'ALLO',
 'DISH',
 'DOCU',
 'WW',
 'HEAR',
 'MVIS',
 'EVGO',
 'UNIT',
 'CLNE',
 'SJ',
 'FCEL',
 'FAT',
 'APP',
 'ROKU',
 'PLUG',
 'NEXT',
 'SAVA',
 'HST',
 'AOSL',
 'MELI',
 'CRSR',
 'GFS',
 'PARA',
 'TECH',
 'NICE',
 'QCOM',
 'ABNB',
 'PDD',
 'SLDP',
 'KIDS',
 'UPST',
 'CSX',
 'DISA',
 'CAMT',
 'PTC',
 'PTRA',
 'ROIC',
 'HOWL',
 'BLNK',
 'AVPT',
 'TIL',
 'AMAT',
 'LYFT',
 'CSCO',
 'PEP']

In [ ]:
top_100 = {}
for symbol in top_picks:
    top_100[symbol] = symbols[symbol]

In [ ]:
# Number of mentions per stock
df = pd.DataFrame(top_100.items()).rename(columns = {0:'Stocks',1:'Number_of_mentions'})
df

,Stocks,Number_of_mentions
0,GME,233
1,TSLA,159
2,AMD,70
3,NVDA,47
4,BBBY,46
...,...,...
95,TIL,1
96,AMAT,1
97,LYFT,1
98,CSCO,1


In [ ]:
# Add sector information to top 100 trending reddit stocks
yfinance = pd.read_csv('../Datasets/Yahoo Finance Collated Stock Information/Yahoo Finance.csv')
sectors = yfinance[['Ticker','Sector']].drop_duplicates()
len(sectors['Ticker'])

98

In [ ]:
df = pd.merge(left =df,right=sectors, left_on = 'Stocks', right_on = 'Ticker').drop(columns = ['Stocks'])

In [ ]:
# df.to_csv('../Datasets/Reddit Data/Ticker Popularity.csv', index = False)

In [ ]:
#create figure object
fig = px.treemap(df, path=[px.Constant("Number of Mentions"),'Sector','Ticker'], values='Number_of_mentions', color='Sector',
hover_data = ['Number_of_mentions','Sector','Ticker'])
fig.update_traces(root_color="lightgrey")
fig.update_layout(margin = dict(t=50, l=25, r=25, b=25))
fig.show()

C:\Users\User\anaconda3\lib\site-packages\plotly\express\_core.py:1616: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\User\anaconda3\lib\site-packages\plotly\express\_core.py:1616: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\User\anaconda3\lib\site-packages\plotly\express\_core.py:1616: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



In [ ]:
with open(r'toptrending_100_stocks.txt', 'w') as fp:
    for item in top_picks:
        # write each item on a new line
        fp.write("%s\n" % item)
    print('Done')

Done
